In [ ]:
# Import packages
import os
from matplotlib import pyplot as plt
import pandas as pd
import datetime
import numpy as np

# Import AuTuMN modules
from autumn.settings import Models, Region
from autumn.settings.folders import OUTPUT_DATA_PATH
from autumn.tools.project import get_project
from autumn.tools import db

from autumn.tools.plots.plotter.base_plotter import COLOR_THEME
from autumn.tools.plots.calibration.plots import get_posterior, get_epi_params, plot_posterior
from autumn.calibration.utils import get_uncertainty_df

In [ ]:

# Specify model details
model = Models.COVID_19
region = Region.SRI_LANKA
dirname = "2022-04-05"

In [ ]:
# get the relevant project and output data
project = get_project(model, region)
project_calib_dir = os.path.join(
    OUTPUT_DATA_PATH, "calibrate", project.model_name, project.region_name
)
calib_path = os.path.join(project_calib_dir, dirname)
# Load tables
mcmc_tables = db.load.load_mcmc_tables(calib_path)
mcmc_params = db.load.load_mcmc_params_tables(calib_path)

In [ ]:
def new_get_posterior(mcmc_params, mcmc_tables, param_name, burn_in=0):
    weighted_vals = []
    for param_df, run_df in zip(mcmc_params, mcmc_tables):
        table_df = param_df.merge(run_df, left_index=True,  right_index=True)
        unweighted_vals = table_df[param_name]
        weights = table_df.weight
        for v, w in zip(unweighted_vals, weights):
            weighted_vals += [v] * w

    return pd.DataFrame(weighted_vals, columns=[param_name])


In [ ]:
#get the quantiles of posterior distributions
param_names = list(mcmc_params[0].columns)
burn_in = 500
quantile_value = [0.025, 0.5, 0.975]
param_list = []

for param_name in param_names:
    posterior_val = new_get_posterior(mcmc_params, mcmc_tables, param_name, burn_in)[param_name]
    quantile_val = np.quantile(posterior_val,quantile_value)
    #print(param_name + f"", quantile_val)
    param_list.append([param_name, quantile_val [0],quantile_val [1], quantile_val [2]])

df = pd.DataFrame(param_list, columns=['param_name', 'quantile 0.025', 'quantile 0.5', 'quantile 0.975'])
df.to_csv("C:/Users/pjay0011/Desktop/posterior_quantiles.csv")


In [ ]:
#plot the posterior distributions
num_bins = 16

for param_name in param_names:
    fig = plt.figure(figsize=(10,8))
    axis = fig.add_subplot(1, 1, 1)
    posterior_val = new_get_posterior(mcmc_params, mcmc_tables, param_name, burn_in)[param_name]
    posterior_val.hist(bins=num_bins, ax=axis, density=True)
    if param_name == "voc_emergence.alpha_beta.contact_rate_multiplier":
        vals_df_alpha = new_get_posterior(mcmc_params, mcmc_tables, param_name, burn_in)[param_name]
        
    if param_name == "voc_emergence.delta.contact_rate_multiplier":
        vals_df_delta = new_get_posterior(mcmc_params, mcmc_tables, param_name, burn_in)[param_name]

In [ ]:
#plot the ratio of Delta and Alpha contact rate multiplier
fontsize = 24
fig = plt.figure(figsize=(10,8))
axis = fig.add_subplot(1, 1, 1)

VoC_contact_rate_ratio =vals_df_delta/vals_df_alpha
num_bins = 16
VoC_contact_rate_ratio.hist(bins=num_bins, ax=axis, density=True, color = "red", ec="red")

axis.spines['top'].set_visible(False)
axis.spines['right'].set_visible(False)
axis.set_xlabel("ratio of Delta:Alpha contact rate multiplier", fontsize = fontsize)
axis.set_ylabel("frequency", fontsize = fontsize)
axis.tick_params(axis='both', labelsize=fontsize)
axis.grid(False)
fig.tight_layout